<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# Tutorial: Helper Functions

## Authors: Samuel Cupp

<font color='red'>**This module is currently under development**</font>

## In this tutorial module we explain several very simple functions provided by the `Con2Prim` gem.

### Required and recommended citations:
* **(Required)** Etienne, Z. B., Paschalidis, V., Haas R., Mösta P., and Shapiro, S. L. IllinoisGRMHD: an open-source, user-friendly GRMHD code for dynamical spacetimes. Class. Quantum Grav. 32 (2015) 175009. ([arxiv:1501.07276](http://arxiv.org/abs/1501.07276)).
* **(Required)** Noble, S. C., Gammie, C. F., McKinney, J. C., Del Zanna, L. Primitive Variable Solvers for Conservative General Relativistic Magnetohydrodynamics. Astrophysical Journal, 641, 626 (2006) ([astro-ph/0512420](https://arxiv.org/abs/astro-ph/0512420)).
* **(Recommended)** Del Zanna, L., Bucciantini N., Londrillo, P. An efficient shock-capturing central-type scheme for multidimensional relativistic flows - II. Magnetohydrodynamics. A&A 400 (2) 397-413 (2003). DOI: 10.1051/0004-6361:20021641 ([astro-ph/0210618](https://arxiv.org/abs/astro-ph/0210618)).

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows

1. [Step 1](#introduction): **Introduction**
1. [Step 2](#guess_primitives): **The `guess_primitives` function**
1. [Step 3](#initialize_diagnostics): **The `initialize_diagnostics` function**
1. [Step 4](#limit_v_and_output_u0): **The `limit_v_and_output_u0` function**
1. [Step 5](#reset_prims_to_atmosphere): **The `reset_prims_to_atmosphere` function**
1. [Step 6](#undensitize_conservatives): **The `undensitize_conservatives` function**
1. [Step 7](#latex_pdf_output): **Output this notebook to $\LaTeX$-formatted PDF file**

<a id='introduction'></a>

# Step 1: Introduction \[Back to [top](#toc)\]
$$\label{introduction}$$

In this tutorial notebook we discuss our code for several helper functions provided by the Con2Prim gem which are too small to need an entire notebook. The functions included in this notebook are
- `guess_primitives`
- `initialize_diagnostics`
- `limit_v_and_output_u0`
- `reset_prims_to_atmosphere`
- `undensitize_conservatives`

All of these use the `GRHayL` structs to pass data.

<a id='guess_primitives'></a>

# Step 2: The `guess_primitives` function \[Back to [top](#toc)\]

$$\label{guess_primitives}$$


First, let's look at the arguments of `guess_primitives`:
```
void guess_primitives(
      const eos_parameters *restrict eos,
      const metric_quantities *restrict metric,
      const primitive_quantities *restrict prims,
      const conservative_quantities *restrict cons,
      primitive_quantities *restrict prims_guess)
```
The first argument appears in most GRHayL functions and contains information on the equation-of-state parameters. The next struct contains (pointwise) information on the metric. Whenever packing the metric struct, we recommend using the function `GRHayL_enforce_detgtij_and_initialize_metric`, which also enforces that the determinant of the conformal metric equals one. GRHayL functions may behave unpredictably with abnormal metrics, as they have not been tested as rigorously outside the expected input ranges.

The outputs are initial guesses for the conservative-to-primitive (C2P) routines. The chosen initial guess sets
\begin{align}
\rho_b &= \frac{\tilde{\rho}}{\psi^6} \\
v^i &= -\beta^i
\end{align}
where the velocities are chosen such that the velocity $\tilde{u}$ used by the C2P routines is zero. Then, the pressure is set to $P_{\rm cold}$ using the function `eos->hybrid_compute_P_cold`. The `guess_primitives` function always computes a guess for the electron fraction and temperature even though this is only needed for the tabulated EOS because its no costlier than including an `if` statement. The electron fraction is computed from the conservatives
$$
Y_e = \frac{\tilde{Y_e}}{\tilde{\rho}}
$$
and the temperature is set to the atmospheric value `eos->T_atm`.

<a id='initialize_diagnostics'></a>

# Step 3: The `initialize_diagnostics` function \[Back to [top](#toc)\]

$$\label{initialize_diagnostics}$$


The function
```
void initialize_diagnostics(con2prim_diagnostics *restrict diagnostics)
```
is very simple. This function just sets all the diagnostic variables in the struct to zero. This function should be call when new instance of this struct is declared or if a `con2prim_diagnostics` instance is reused.

<a id='limit_v_and_output_u0'></a>

# Step 4: The `limit_v_and_output_u0` function \[Back to [top](#toc)\]
$$\label{limit_v_and_output_u0}$$


First, let's look at the arguments of `limit_v_and_output_u0`:
```
void limit_v_and_output_u0(
      const eos_parameters *restrict eos,
      const metric_quantities *restrict metric,
      primitive_quantities *restrict prims,
      double *restrict u0_out,
      con2prim_diagnostics *restrict diagnostics) 
```

The outputs are the speed-limited $v^i$ and $u^0$.

First, we need to find a good inequality to use to limit $v$.
\begin{align}
\frac{\gamma_{ij} (v^i + \beta^i)(v^j + \beta^j)}{\alpha^2} & = \frac{\gamma_{ij}}{(u^0\alpha)^2} ( \gamma^{ik} u_k \gamma^{jl} u_l) \\
%
& = \frac{1}{(u^0\alpha)^2} u_j u_l \gamma^{jl} \\
%
& = \frac{1}{(u^0\alpha)^2} ( (u^0 \alpha)^2 - 1 ) \\
& = \frac{1}{(u^0\alpha)^2} <= 1
\end{align}
The first step uses eq. 53 of arXiv:astro-ph/0503420. The second step follows from the identity\
$$\gamma_{ij} \gamma^{ik} = \delta^k_j$$
The third step uses eq. 56 of arXiv:astro-ph/0503420. From this, we can set a threshold above which the speed limiter triggers. This limit is based on the parameter $W_\mathrm{max}$, the maximum allowed Lorentz factor for the simulation. Defining the quantities
$$ L_\mathrm{in} = \frac{1}{(u^0\alpha)^2} $$
$$ L_\mathrm{max} = \frac{1}{W_\mathrm{max}^2} $$
Then, the condition for limiting the speed is just
$$ L_\mathrm{in} > L_\mathrm{max} $$
The speed limiter then rescales the velocities using
$$v^i = \frac{L_\mathrm{max}}{L_\mathrm{in}}\left( v^i + \beta^i \right) - \beta^i$$

The final step is to calculate $u^0$. Since we have the lapse from the metric struct, we can simply rearrange $L_\mathrm{in}$ (or $L_\mathrm{max}$ if speed limited) to get $u^0$.

<a id='reset_prims_to_atmosphere'></a>

# Step 5: The `reset_prims_to_atmosphere` function \[Back to [top](#toc)\]

$$\label{reset_prims_to_atmosphere}$$


First, let's look at the arguments of `reset_prims_to_atmosphere`:
```
void reset_prims_to_atmosphere(
      const GRHayL_parameters *restrict params,
      const eos_parameters *restrict eos,
      const metric_quantities *restrict metric,
      primitive_quantities *restrict prims)
```
The first two arguments appear in most GRHayL functions and contain information on the simulation and equation-of-state parameters. The next struct contains (pointwise) information on the metric. Whenever packing the metric struct, we recommend using the function `GRHayL_enforce_detgtij_and_initialize_metric`, which also enforces that the determinant of the conformal metric equals one. GRHayL functions may behave unpredictably with abnormal metrics, as they have not been tested as rigorously outside the expected input ranges.

The outputs are the primitives.

This function is quite simple. All the primitive values are simply set to the atmospheric values stored in the `eos` struct. The velocities are not set via this prescription, so they are set to a value of 0.

<a id='undensitize_conservatives'></a>

# Step 6: The `undensitize_conservatives` function \[Back to [top](#toc)\]

$$\label{undensitize_conservatives}$$


First, let's look at the arguments of `undensitize_conservatives`:
```
void undensitize_conservatives(
      const metric_quantities *restrict metric,
      const conservative_quantities *restrict cons,
      conservative_quantities *restrict cons_undens)
```
The first struct contains (pointwise) information on the metric. Whenever packing the metric struct, we recommend using the function `GRHayL_enforce_detgtij_and_initialize_metric`, which also enforces that the determinant of the conformal metric equals one. GRHayL functions may behave unpredictably with abnormal metrics, as they have not been tested as rigorously outside the expected input ranges. The second input points to a `conservative_quantities` struct with densitized conservative variables.

The outputs are undensitized versions of the inputs stored in `cons_undens`. These are calculated by dividing all the `cons` elements by `metric->psi6`.

<a id='latex_pdf_output'></a>

# Step 7: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.pdf](Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means).

latex_nrpy_style_path = os.path.join(nrpy_dir_path,"latex_nrpy_style.tplx")
#!jupyter nbconvert --to latex --template $latex_nrpy_style_path --log-level='WARN' Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.ipynb
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.tex
#!pdflatex -interaction=batchmode Tutorial-IllinoisGRMHD__apply_tau_floor__enforce_limits_on_primitives_and_recompute_conservs.tex
!rm -f Tut*.out Tut*.aux Tut*.log